In [155]:
# import os
import re
import pymupdf
import pandas as pd

In [156]:
file_path = "handbook.pdf"

In [157]:
print(pymupdf.__version__)

1.26.3


In [158]:
from tqdm.auto import tqdm

In [5]:
!pip install tqdm


[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [169]:
def read_pdf(file_path):
    doc = pymupdf.open(file_path)
    pages_and_texts = []

    for page_number , page in enumerate(tqdm(doc, desc="Reading Pages....")):
        text = page.get_text()
        text = text.replace("\n", " ").strip()
        # text = text.split(" ")[1:]
        # text = " ".join(text)
        pages_and_texts.append({
            "page number": page_number - 11,
            "number of characters": len(text),
            "word count": len(text.split(" ")),
            "page_token_count": len(text) / 4,
            "text": text
        })

    return pages_and_texts

pages_and_texts = read_pdf(file_path)
    

Reading Pages....:   4%|▎         | 7/192 [00:00<00:03, 60.87it/s]

Reading Pages....: 100%|██████████| 192/192 [00:00<00:00, 195.66it/s]


In [170]:
pages_and_texts

[{'page number': -11,
  'number of characters': 0,
  'word count': 1,
  'page_token_count': 0.0,
  'text': ''},
 {'page number': -10,
  'number of characters': 62,
  'word count': 19,
  'page_token_count': 15.5,
  'text': 'COVENANT UNIVERSITY  STUDENT HANDBOOK            (2019 – 2022)'},
 {'page number': -9,
  'number of characters': 335,
  'word count': 83,
  'page_token_count': 83.75,
  'text': 'i  Covenant University  Student Handbook | 2019 - 2022    The Board of Regents of Covenant University reserves the right to alter any  section, provision, or clause of this document as it deems fit from time to  time.                    Published by   Covenant University,   KM 10, Idiroko Road,   P.M.B. 1023, Ota, Ogun State, Nigeria'},
 {'page number': -8,
  'number of characters': 482,
  'word count': 105,
  'page_token_count': 120.5,
  'text': 'ii  CHANCELLOR  David O. Oyedepo, Ph.D., FNAE    PRINCIPAL OFFICERS  VICE-CHANCELLOR Prof. AAA. Atayero      DEPUTY VICE-CHANCELLOR Prof. Akan Will

In [171]:
df = pd.DataFrame(pages_and_texts)
df

,page number,number of characters,word count,page_token_count,text
0,-11,0,1,0.00,
1,-10,62,19,15.50,COVENANT UNIVERSITY STUDENT HANDBOOK ...
2,-9,335,83,83.75,i Covenant University Student Handbook | 201...
3,-8,482,105,120.50,"ii CHANCELLOR David O. Oyedepo, Ph.D., FNAE ..."
4,-7,2438,188,609.50,iii TABLE OF CONTENTS Table of Contents .....
...,...,...,...,...,...
187,176,727,150,181.75,176 19 Pairing Three (3) weeks suspension i...
188,177,236,49,59.00,177 30 Breach of University Peace Ranging...
189,178,706,153,176.50,178 NATIONAL ANTHEM Arise O' compatriots Ni...
190,179,629,132,157.25,179 COVENANT UNIVERSITY ANTHEM We're a Coven...


In [172]:
# Get stats
df.describe().round(2)

,page number,number of characters,word count,page_token_count
count,192.00,192.00,192.00,192.00
mean,84.50,1101.18,185.34,275.30
std,55.57,481.36,60.11,120.34
min,-11.00,0.00,1.00,0.00
25%,36.75,886.75,156.75,221.69
50%,84.50,1113.50,203.00,278.38
75%,132.25,1321.25,227.00,330.31
max,180.00,3051.00,291.00,762.75


In [173]:
# mean token count = 275.30
# so I can use a 384 embedding model!

I want to split each page into sections and subsections

In [174]:
pattern = re.compile(
    r"(\d+(?:\.\d+)*)(?:\.\s*|\s+)([A-Z][A-Z\s:\-\(\)]+?)(?=(?:\s+[A-Z][a-z])|(?:\s*\d+\.)|$)"
)

def split_sections(text):
    matches = list(pattern.finditer(text))
    sections = []
    
    for i, match in enumerate(matches):
        title = match.string[match.start():match.end()].strip()
        start = match.end()
        end = matches[i+1].start() if i+1 < len(matches) else len(text)
        content = text[start:end].strip()
        
        sections.append({
            "title": title,
            "text": content
        })
    return sections

In [175]:
document = []
current_chapter = ""
for page in pages_and_texts[11:]:
    sections = split_sections(page["text"])

    if sections == []:
        continue
    
    document.append(sections)

In [176]:
document

[[{'title': '1  CHAPTER ONE  INTRODUCTION', 'text': ''},
  {'title': '1.1.  THE NAME: COVENANT UNIVERSITY (CU)',
   'text': "All over Africa, and Nigeria in particular, a great significance is  attached to names. They portray meanings and convey important  messages. Names reflect circumstances of birth or events. The  word “Covenant” was chosen as an expression of the University's  total commitment and vow to make a Total Man of her students. It  reflects the intention of the proprietors of the University to uphold  a binding agreement with students to deliver their desires for  excellence and career exploits by offering them the best in  educational attainment and their parents/guardians the best"}],
 [{'title': '1.2.  THE UNIVERSITY',
   'text': 'On October 21, 2002, the African educational landscape was  radically altered by the formal entry of Covenant University (CU)  into the higher education context. The University is located at  Canaanland, Ota, Ogun State, Nigeria. The Univers

In [177]:
pattern = r"\s+CHAPTER\s+(\S+)\s+(\S+)"
current_chapter = ""
book_dict = []
for doc in document:
    for dict in doc:
        # print(dict)
        result = re.search(pattern, dict["title"])
        if dict["text"] == '' or result:
            current_chapter = dict["title"]
            current_chapter =  current_chapter.split(" ")[2:]
            current_chapter =  " ".join(current_chapter)
            continue
        
        print(current_chapter)
        dict["chapter"] = current_chapter

        
        book_dict.append(dict)


CHAPTER ONE  INTRODUCTION
CHAPTER ONE  INTRODUCTION
CHAPTER ONE  INTRODUCTION
CHAPTER ONE  INTRODUCTION
CHAPTER ONE  INTRODUCTION
CHAPTER ONE  INTRODUCTION
CHAPTER ONE  INTRODUCTION
CHAPTER ONE  INTRODUCTION
CHAPTER ONE  INTRODUCTION
CHAPTER TWO  COVENANT UNIVERSITY OPERATIONAL ARMS
CHAPTER TWO  COVENANT UNIVERSITY OPERATIONAL ARMS
CHAPTER TWO  COVENANT UNIVERSITY OPERATIONAL ARMS
CHAPTER TWO  COVENANT UNIVERSITY OPERATIONAL ARMS
CHAPTER TWO  COVENANT UNIVERSITY OPERATIONAL ARMS
CHAPTER TWO  COVENANT UNIVERSITY OPERATIONAL ARMS
CHAPTER TWO  COVENANT UNIVERSITY OPERATIONAL ARMS
CHAPTER TWO  COVENANT UNIVERSITY OPERATIONAL ARMS
CHAPTER TWO  COVENANT UNIVERSITY OPERATIONAL ARMS
CHAPTER TWO  COVENANT UNIVERSITY OPERATIONAL ARMS
CHAPTER TWO  COVENANT UNIVERSITY OPERATIONAL ARMS
CHAPTER TWO  COVENANT UNIVERSITY OPERATIONAL ARMS
CHAPTER TWO  COVENANT UNIVERSITY OPERATIONAL ARMS
CHAPTER THREE  COVENANT UNIVERSITY AT A GLANCE
CHAPTER THREE  COVENANT UNIVERSITY AT A GLANCE
CHAPTER THREE  COVENAN

In [178]:
book_dict

[{'title': '1.1.  THE NAME: COVENANT UNIVERSITY (CU)',
  'text': "All over Africa, and Nigeria in particular, a great significance is  attached to names. They portray meanings and convey important  messages. Names reflect circumstances of birth or events. The  word “Covenant” was chosen as an expression of the University's  total commitment and vow to make a Total Man of her students. It  reflects the intention of the proprietors of the University to uphold  a binding agreement with students to deliver their desires for  excellence and career exploits by offering them the best in  educational attainment and their parents/guardians the best",
  'chapter': 'CHAPTER ONE  INTRODUCTION'},
 {'title': '1.2.  THE UNIVERSITY',
  'text': 'On October 21, 2002, the African educational landscape was  radically altered by the formal entry of Covenant University (CU)  into the higher education context. The University is located at  Canaanland, Ota, Ogun State, Nigeria. The University is a growing,  d